
## A Practical Guide to Building LLM Applications

This notebook provides a hands-on introduction to LangChain, covering essential concepts and practical implementations for working with large language models.


## 1. Environment Setup & Configuration

First, let's load environment variables and set up our workspace:

In [7]:
from IPython.display import Markdown
from dotenv import load_dotenv
load_dotenv()  # Loads API keys from .env file

True

**Key Concepts:**
- `load_dotenv()` reads environment variables from a `.env` file
- This keeps API keys secure and out of your code
- Required for authentication with LLM providers (OpenAI, Google, etc.)



## 2. Understanding LLMs and Chat Models

### What are Large Language Models (LLMs)?
LLMs are advanced neural networks trained on massive text datasets using transformer architecture. They predict the next token based on context, enabling human-like text generation.

### LangChain's Abstraction Layer
LangChain provides a unified interface for different LLM providers because:
- Different APIs have different authentication methods and parameters
- Unified interface allows switching models with minimal code changes
- Enables building complex chains that work across providers

###  Chat Models vs Raw LLMs
| Feature | Chat Models | Raw LLMs |
|---------|-------------|----------|
| **Optimization** | Multi-turn conversations | Text completion |
| **Message Structure** | System/User/Assistant roles | Plain text |
| **Context** | Maintains conversation history | Stateless |
| **Best for** | Dialogues, assistants | Text generation, summarization |


## 3. Integrating Different LLM Providers

### 3.1 OpenAI Integration

In [ ]:
from langchain_openai import ChatOpenAI


openai_llm = ChatOpenAI(
    model_name="gpt-4o",   # Model version
    temperature=0,         # Controls randomness (0-2)
    # max_tokens=100,      # Optional: limit response length
    # timeout=30,          # Optional: request timeout
)

response = openai_llm.invoke("What is the capital of France?")
print(response)

**Temperature Parameter Explained:**
- **0.0-0.3**: Deterministic, consistent responses
- **0.4-0.7**: Balanced creativity and consistency
- **0.8-1.2**: Creative, varied responses
- **1.3-2.0**: Highly creative, less predictable

### 3.2 Google Gemini Integration

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

google_llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=1.0,      # More creative responses
    max_tokens=None,      # No token limit
    timeout=None,         # No timeout
    max_retries=2,        # Automatic retry on failure
)

response = google_llm.invoke("what is ai?")
print(response)

### 3.3 HuggingFace Integration

In [ ]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

# Using HuggingFace's hosted inference API
hf_llm = HuggingFaceEndpoint(
    repo_id="deepseek-ai/DeepSeek-V3.2",
    task="text-generation",
    max_new_tokens=512, 
)

chat_model = ChatHuggingFace(llm=hf_llm)
response = chat_model.invoke("What is the capital of Italy?")
print(response)

**Benefits of Open-Source Models:**
- **No API Costs**: Run locally or on your infrastructure
- **Data Privacy**: No external API calls for sensitive data
- **Customization**: Fine-tune for specific domains
- **Transparency**: Full control over model behavior

## 4. Working with Embeddings

### What are Embeddings?
Embeddings convert text into numerical vectors that capture semantic meaning. Similar concepts have similar vectors.

In [ ]:
from langchain_openai import OpenAIEmbeddings

# Initialize embeddings model
openai_embedding = OpenAIEmbeddings(model="text-embedding-3-small")

# Convert text to vector
vector = openai_embedding.embed_query("Hello world")
print(f"Vector dimension: {len(vector)}") 


**Understanding Embeddings:**
- Each word/sentence becomes a point in high-dimensional space
- Similar meanings = Closer points in vector space
- Enables mathematical operations on text (similarity, clustering, etc.)

**Practical Applications:**
1. **Semantic Search**: Find documents by meaning, not keywords
2. **RAG Systems**: Retrieve relevant context for LLM queries
3. **Clustering**: Group similar documents automatically
4. **Recommendations**: Suggest similar items based on content

## 5. Crafting Effective Prompts

### 5.1 Basic Prompt Templates

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

google_llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=1.0,      # More creative responses
    max_tokens=None,      # No token limit
    timeout=None,         # No timeout
    max_retries=2,        # Automatic retry on failure
)

# Create reusable template
TEMPLATE = """ 
What is the capital of {country}?
Also provide some {additional_info}
"""

prompt_template = PromptTemplate(
    template=TEMPLATE,
    input_variables=["country", "additional_info"]
)

# Format the template
formatted_prompt = prompt_template.format(
    country="Italy",
    additional_info="tourist attractions"
)
print(formatted_prompt)
response = google_llm.invoke(formatted_prompt)
print(response)


**Why Use Templates?**
- **Consistency**: Standardized prompt structure
- **Reusability**: Use same template across different inputs
- **Maintainability**: Update prompts in one place
- **Testing**: Easy to test different prompt variations
- **Validation**: Provides built in validation

### 5.2 The Stateless Problem

In [ ]:
response1 = google_llm.invoke("Hello! I am Sandesh Dhital")
print(response1.content)

response2 = google_llm.invoke("Can you tell my name?")
print(response2.content)  # LLM doesn't remember!

**The Challenge:**
- Each API call is independent
- No memory between requests
- Real conversations need context

**The Solution:**
- Maintain message history


In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# Initialize conversation
message_history = [
    SystemMessage(content="You are a helpful assistant."),
]

# Simulate conversation
messages = [
    HumanMessage(content="Hi, I'm Aashik"),
    AIMessage(content="Hello Aashik! How can I help?"),
    HumanMessage(content="Why is python considered beginner friendly?"),
]

# Send entire history
response = google_llm.invoke(message_history + messages)
print(response.content)

### 5.3 Multi-Turn Conversations



**Message Types:**
- **SystemMessage**: Sets behavior/role (invisible to user)
- **HumanMessage**: User inputs/questions
- **AIMessage**: Model responses (for context)

**Best Practices:**
1. Start with a clear SystemMessage
2. Append all messages to history
3. Consider token limits (long histories = more tokens)
4. Use summarization for very long conversations

### 5.4 Advanced Chat Prompt Templates

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# Structured chat template with roles
chat_template = ChatPromptTemplate([
    ('system', 'You are a helpful {domain} expert'),
    ('human', 'Explain in simple terms, what is {topic}')
])

# Format with variables
prompt = chat_template.invoke({
    'domain': 'cricket',
    'topic': 'NPL'
})
print(prompt)


## 6. Structured Output Generation

### 6.1 Using TypedDict for Structure

In [9]:
from typing import TypedDict, Annotated, Optional, Literal

# Define expected output structure
class Review(TypedDict):
    key_themes: Annotated[list[str], "Key themes in the review"]
    summary: Annotated[str, "Brief summary"]
    sentiment: Annotated[Literal["pos", "neg"], "Sentiment"]
    pros: Annotated[Optional[list[str]], "List of pros"]
    cons: Annotated[Optional[list[str]], "List of cons"]
    name: Annotated[Optional[str], "Name of the person who had reviewed"]

# Configure LLM for structured output
structured_llm = google_llm.with_structured_output(Review)

review="""
I recently upgraded to the Samsung Galaxy S24 Ultra, and I must say, it’s an absolute powerhouse! The Snapdragon 8 Gen 3 processor makes everything lightning fast—whether I’m gaming, multitasking, or editing photos. The 5000mAh battery easily lasts a full day even with heavy use, and the 45W fast charging is a lifesaver.

The S-Pen integration is a great touch for note-taking and quick sketches, though I don't use it often. What really blew me away is the 200MP camera—the night mode is stunning, capturing crisp, vibrant images even in low light. Zooming up to 100x actually works well for distant objects, but anything beyond 30x loses quality.

However, the weight and size make it a bit uncomfortable for one-handed use. Also, Samsung’s One UI still comes with bloatware—why do I need five different Samsung apps for things Google already provides? The $1,300 price tag is also a hard pill to swallow.

Pros:
Insanely powerful processor (great for gaming and productivity)
Stunning 200MP camera with incredible zoom capabilities
Long battery life with fast charging
S-Pen support is unique and useful
                                 
Review by Sandesh Dhital"""

# Get structured response
result = structured_llm.invoke(review)
print(result)

{'key_themes': ['processor performance', 'camera quality', 'battery life', 'S-Pen', 'design and ergonomics', 'software bloatware', 'price'], 'summary': 'The Samsung Galaxy S24 Ultra offers exceptional performance with its Snapdragon 8 Gen 3 processor, a stunning 200MP camera with great night mode and zoom capabilities, and a long-lasting battery with fast charging. The S-Pen is a useful addition. However, its size and weight can be a drawback for one-handed use, One UI has bloatware, and the price is high.', 'sentiment': 'pos', 'pros': ['Insanely powerful Snapdragon 8 Gen 3 processor, great for gaming and productivity', 'Stunning 200MP camera with incredible night mode and good zoom capabilities up to 30x', 'Long-lasting 5000mAh battery with 45W fast charging', 'S-Pen support is unique and useful for note-taking and sketches'], 'cons': ['Weight and size make it uncomfortable for one-handed use', "Samsung's One UI still comes with bloatware", 'High price tag of $1,300', 'Zoom beyond 30x

**Why Structured Output?**
- **Reliable Parsing**: No regex or manual parsing
- **Type Safety**: Guaranteed data types
- **Validation**: Built-in schema validation
- **Integration**: Easy to use with databases/APIs

In [ ]:
from pydantic import BaseModel, Field

# Define schema with validation
class Review(BaseModel):
    key_themes: list[str] = Field(description="Key themes in list")
    summary: str = Field(description="Brief summary")
    sentiment: Literal["pos", "neg"] = Field(description="Sentiment")
    pros: Optional[list[str]] = Field(default=None, description="Pros list")
    cons: Optional[list[str]] = Field(default=None, description="Cons list")
    name: Optional[str] = Field(default=None, description="Reviewer name")

# Use with LLM
structured_llm = openai_llm.with_structured_output(Review)

review="""
I recently upgraded to the Samsung Galaxy S24 Ultra, and I must say, it’s an absolute powerhouse! The Snapdragon 8 Gen 3 processor makes everything lightning fast—whether I’m gaming, multitasking, or editing photos. The 5000mAh battery easily lasts a full day even with heavy use, and the 45W fast charging is a lifesaver.

The S-Pen integration is a great touch for note-taking and quick sketches, though I don't use it often. What really blew me away is the 200MP camera—the night mode is stunning, capturing crisp, vibrant images even in low light. Zooming up to 100x actually works well for distant objects, but anything beyond 30x loses quality.

However, the weight and size make it a bit uncomfortable for one-handed use. Also, Samsung’s One UI still comes with bloatware—why do I need five different Samsung apps for things Google already provides? The $1,300 price tag is also a hard pill to swallow.

Pros:
Insanely powerful processor (great for gaming and productivity)
Stunning 200MP camera with incredible zoom capabilities
Long battery life with fast charging
S-Pen support is unique and useful
                                 
Review by Sandesh Dhital"""

# Get structured response
result = structured_llm.invoke(review)
print(result)




**Pydantic Advantages:**
- **Runtime Validation**: Catches invalid data
- **Rich Features**: Default values, validators, serialization
- **Better Integration**: Works with FastAPI, databases
- **Documentation**: Auto-generated API docs

### 6.3 Using JSON Schema (Maximum Flexibility)

In [1]:
# schema
json_schema = {
  "title": "Review",
  "type": "object",
  "properties": {
    "key_themes": {
      "type": "array",
      "items": {
        "type": "string"
      },
      "description": "Write down all the key themes discussed in the review in a list"
    },
    "summary": {
      "type": "string",
      "description": "A brief summary of the review"
    },
    "sentiment": {
      "type": "string",
      "enum": ["pos", "neg"],
      "description": "Return sentiment of the review either negative, positive or neutral"
    },
    "pros": {
      "type": ["array", "null"],
      "items": {
        "type": "string"
      },
      "description": "Write down all the pros inside a list"
    },
    "cons": {
      "type": ["array", "null"],
      "items": {
        "type": "string"
      },
      "description": "Write down all the cons inside a list"
    },
    "name": {
      "type": ["string", "null"],
      "description": "Write the name of the reviewer"
    }
  },
  "required": ["key_themes", "summary", "sentiment"]
}


structured_llm = openai_llm.with_structured_output(json_schema)

review="""
I recently upgraded to the Samsung Galaxy S24 Ultra, and I must say, it’s an absolute powerhouse! The Snapdragon 8 Gen 3 processor makes everything lightning fast—whether I’m gaming, multitasking, or editing photos. The 5000mAh battery easily lasts a full day even with heavy use, and the 45W fast charging is a lifesaver.

The S-Pen integration is a great touch for note-taking and quick sketches, though I don't use it often. What really blew me away is the 200MP camera—the night mode is stunning, capturing crisp, vibrant images even in low light. Zooming up to 100x actually works well for distant objects, but anything beyond 30x loses quality.

However, the weight and size make it a bit uncomfortable for one-handed use. Also, Samsung’s One UI still comes with bloatware—why do I need five different Samsung apps for things Google already provides? The $1,300 price tag is also a hard pill to swallow.

Pros:
Insanely powerful processor (great for gaming and productivity)
Stunning 200MP camera with incredible zoom capabilities
Long battery life with fast charging
S-Pen support is unique and useful
                                 
Review by Sandesh Dhital"""

# Get structured response
result = structured_llm.invoke(review)
print(result)


NameError: name 'openai_llm' is not defined

**JSON Schema Benefits:**
- **Language Agnostic**: Works across programming languages
- **Standard Format**: Widely supported in APIs
- **Complex Constraints**: Advanced validation rules


# 7.LangChain Output Parsers 


This section provides an in-depth guide to LangChain's output parsers, covering `StrOutputParser`, `JsonOutputParser`, `StructuredOutputParser`, and `PydanticOutputParser` with practical examples.





## 7.1 Introduction to Output Parsers

### Why Output Parsers Matter
LLMs generate unstructured text, but real applications need structured data. Output parsers solve this by:
- **Extracting structured data** from text responses
- **Validating** outputs against schemas
- **Normalizing** inconsistent formats
- **Enabling** reliable downstream processing



## 7.2 StrOutputParser - Simple Text Extraction

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Initialize components
llm = ChatOpenAI(model="gpt-4o", temperature=0.7)
parser = StrOutputParser()

# Create simple chain
prompt = PromptTemplate.from_template("Summarize this text: {text}")
chain = prompt | llm | parser

# Execute
text = "Large Language Models (LLMs) are AI systems trained on vast amounts of text data..."
result = chain.invoke({"text": text})

print("StrOutputParser Result:")
print(f"Content: {result}")
print(f"Type: {type(result)}")  # <class 'str'>
print(f"Length: {len(result)} characters")


**Key Points:**
- Returns raw string output
- No validation or structure
- Fastest and simplest parser
- Good for text-only responses

## 7.3 JsonOutputParser - Structured JSON Responses

In [2]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

# Initialize JSON parser
parser = JsonOutputParser()

# Create prompt with format instructions
prompt = PromptTemplate(
    template="""Extract the following information as JSON:
    {text}
    
    {format_instructions}
    """,
    input_variables=["text"],
    partial_variables={
        "format_instructions": parser.get_format_instructions()
    }
)

# Create chain
chain = prompt | llm | parser

text = """
Customer: John Smith
Email: john@example.com
Order: #12345
Total: $199.99
Status: Delivered
Date: 2024-03-15
Items: Laptop, Mouse, Keyboard
"""

result = chain.invoke({"text": text})

print("JsonOutputParser Result:")
print(f"Parsed: {result}")
print(f"Type: {type(result)}")  # <class 'dict'>
print(f"Keys: {list(result.keys())}")

NameError: name 'llm' is not defined

**Key Points:**
- Extracts JSON from LLM responses
- Can validate JSON syntax
- Good for structured data exchange
- Works with any JSON-compliant output

## 7.4 StructuredOutputParser - Custom Format Parsing

In [ ]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.prompts import PromptTemplate

# Define output schema
response_schemas = [
    ResponseSchema(name="summary", description="Brief summary"),
    ResponseSchema(name="sentiment", description="Positive/Neutral/Negative"),
    ResponseSchema(name="keywords", description="List of keywords"),
    ResponseSchema(name="confidence", description="Confidence score 0-1", type="float")
]

# Create parser
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()

print("Format Instructions:")
print(format_instructions)

# Create prompt
prompt = PromptTemplate(
    template="""Analyze this text:
    {text}
    
    {format_instructions}
    """,
    input_variables=["text"],
    partial_variables={"format_instructions": format_instructions}
)

# Create chain
chain = prompt | llm | parser

# Execute
text = "The new smartphone features an amazing camera, fast processor, but battery life could be better."
result = chain.invoke({"text": text})

print("\nStructuredOutputParser Result:")
for key, value in result.items():
    print(f"{key}: {value} (type: {type(value).__name__})")

**Key Points:**
- Highly flexible structure definition
- Custom delimiters and patterns
- Supports nested schemas
- Good for complex extraction tasks

## 7.5 PydanticOutputParser - Type-Safe Excellence

In [ ]:

from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

load_dotenv()

# Define the model
llm=ChatOpenAI()


class Person(BaseModel):

    name: str = Field(description='Name of the person')
    age: int = Field(gt=18, description='Age of the person')
    city: str = Field(description='Name of the city the person belongs to')

parser = PydanticOutputParser(pydantic_object=Person)

template = PromptTemplate(
    template='Generate the name, age and city of a fictional {place} person \n {format_instruction}',
    input_variables=['place'],
    partial_variables={'format_instruction':parser.get_format_instructions()}
)

chain = template | llm | parser

final_result = chain.invoke({'place':'sri lankan'})

print(final_result)

**Key Points:**
- Full type safety with runtime validation
- Complex validation rules and computed properties
- Best for production systems requiring reliability
- Excellent integration with Python ecosystem
- Can be combined with other parsers for fallback strategies


###  Useful Resources
- [LangChain Documentation](https://python.langchain.com/)
- [OpenAI API Docs](https://platform.openai.com/docs/)
- [HuggingFace Models](https://huggingface.co/models)
- [Pydantic Documentation](https://docs.pydantic.dev/)



**💡 Remember**: Start simple, test often, and iterate based on results. LLM applications are experimental - expect to adjust prompts and parameters based on your specific use case!

Happy building! 🚀